In [ ]:
import sagemaker

session = sagemaker.Session()

In [ ]:
%%sh
# https://s3.amazonaws.com/amazon-reviews-pds/readme.html
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp
gzip -d /tmp/amazon_reviews_us_Camera_v1_00.tsv.gz

In [ ]:
prefix = 'amazon-reviews-camera'

input_data = session.upload_data(path='/tmp/amazon_reviews_us_Camera_v1_00.tsv', key_prefix=prefix)
print(input_data)

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

role = sagemaker.get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.c5.2xlarge',
                                     instance_count=1)

In [ ]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code='preprocessing-word2vec.py',
    
    inputs=[
        ProcessingInput(
                source=input_data,
                destination='/opt/ml/processing/input')
    ],
    
    outputs=[
        ProcessingOutput(
                output_name='train_data',
                source='/opt/ml/processing/train')
    ],
    
    arguments=[
               #'--num-reviews', '100000',
               '--filename', 'amazon_reviews_us_Camera_v1_00.tsv'
    ]
)

In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])